# Logic

This Jupyter notebook acts as supporting material for topics covered in __Chapter 6 Logical Agents__, __Chapter 7 First-Order Logic__ and __Chapter 8 Inference in First-Order Logic__ of the book *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)*. We make use of the implementations in the [logic.py](https://github.com/aimacode/aima-python/blob/master/logic.py) module. See the [intro notebook](https://github.com/aimacode/aima-python/blob/master/intro.ipynb) for instructions.

Let's first import everything from the `logic` module.

In [ ]:
from utils import *
from logic import *
from notebook import psource

## 📚 CONTENTS & LEARNING ROADMAP

This notebook covers the essential concepts and algorithms for logical reasoning in AI. Here's your learning journey:

### 🏗️ **Foundation: Representing Logic in Code**
- **Expr**: Building blocks for logical expressions 
- **PropKB**: Knowledge bases for propositional logic
- **Knowledge-based agents**: AI agents that reason with logic

### ⚡ **Core Algorithms: Propositional Logic Inference**
- **Truth table enumeration**: The brute-force approach (complete but slow)
- **Proof by resolution**: Elegant contradiction-based reasoning
- **Forward and backward chaining**: Efficient inference for Horn clauses
- **DPLL**: Smart backtracking with pruning heuristics
- **WalkSAT**: Fast local search for satisfiability
- **SATPlan**: Planning through logical constraint satisfaction

### 🎯 **Advanced: First-Order Logic** 
- **FolKB**: Knowledge bases with variables and quantifiers
- **Unification**: Pattern matching for logical expressions
- **FOL Forward chaining**: Data-driven inference with variables
- **FOL Backward chaining**: Goal-driven reasoning with variables

### 🎓 **Learning Tips for Students**
- 💡 **Start Simple**: Master propositional logic before moving to first-order logic
- 🔍 **Understand Trade-offs**: Each algorithm has different strengths (completeness vs. efficiency)
- 🛠️ **Practice**: Try the examples and modify them to build intuition
- 🧠 **Think Applications**: Consider how these techniques apply to real AI problems

## Logical Sentences

The `Expr` class is designed to represent any kind of mathematical expression. The simplest type of `Expr` is a symbol, which can be defined with the function `Symbol`:

In [ ]:
Symbol('x')

Or we can define multiple symbols at the same time with the function `symbols`:

In [ ]:
(x, y, P, Q, f) = symbols('x, y, P, Q, f')

We can combine `Expr`s with the regular Python infix and prefix operators. Here's how we would form the logical sentence "P and not Q":

This works because the `Expr` class overloads the `&` operator with this definition:

```python
def __and__(self, other): return Expr('&',  self, other)```
     
and does similar overloads for the other operators. An `Expr` has two fields: `op` for the operator, which is always a string, and `args` for the arguments, which is a tuple of 0 or more expressions. By "expression," I mean either an instance of `Expr`, or a number. Let's take a look at the fields for some `Expr` examples:

In [ ]:
P & ~Q

In [ ]:
sentence = P & ~Q

sentence.op

In [ ]:
sentence.args

In [ ]:
P.op

In [ ]:
P.args

In [ ]:
Pxy = P(x, y)

Pxy.op

In [ ]:
Pxy.args

**Important Concept**: The `Expr` class does not define the *logic* of Propositional Logic sentences; it just gives you a way to *represent* expressions. Think of an `Expr` as an [abstract syntax tree](https://en.wikipedia.org/wiki/Abstract_syntax_tree) - a tree structure that represents the syntactic structure of your logical expression.

**Key Points to Remember**:
- Each of the `args` in an `Expr` can be either a symbol, a number, or a nested `Expr`
- We can nest these trees to any depth, allowing for complex logical expressions
- The `Expr` class is like a container that holds the structure of your logic, not the meaning

Here is a deeply nested `Expr` example:

In [ ]:
3 * f(x, y) + P(y) / 2 + 1

## Operators for Constructing Logical Sentences

Here is a table of the operators that can be used to form sentences. Note that we have a problem: we want to use Python operators to make sentences, so that our programs (and our interactive sessions like the one here) will show simple code. But Python does not allow implication arrows as operators, so for now we have to use a more verbose notation that Python does allow: `|'==>'|` instead of just `==>`. Alternately, you can always use the more verbose `Expr` constructor forms:

| Operation                | Book | Python Infix Input | Python Output | Python `Expr` Input
|--------------------------|----------------------|-------------------------|---|---|
| Negation                 | &not; P      | `~P`                       | `~P` | `Expr('~', P)`
| And                      | P &and; Q       | `P & Q`                     | `P & Q` | `Expr('&', P, Q)`
| Or                       | P &or; Q | `P`<tt> &#124; </tt>`Q`| `P`<tt> &#124; </tt>`Q` | `Expr('`&#124;`', P, Q)`
| Inequality (Xor)         | P &ne; Q     | `P ^ Q`                | `P ^ Q`  | `Expr('^', P, Q)`
| Implication                  | P &rarr; Q    | `P` <tt>&#124;</tt>`'==>'`<tt>&#124;</tt> `Q`   | `P ==> Q` | `Expr('==>', P, Q)`
| Reverse Implication      | Q &larr; P     | `Q` <tt>&#124;</tt>`'<=='`<tt>&#124;</tt> `P`  |`Q <== P` | `Expr('<==', Q, P)`
| Equivalence            | P &harr; Q   | `P` <tt>&#124;</tt>`'<=>'`<tt>&#124;</tt> `Q`   |`P <=> Q` | `Expr('<=>', P, Q)`

Here's an example of defining a sentence with an implication arrow:

In [ ]:
~(P & Q)  |'==>'|  (~P | ~Q)

## `expr`: a Shortcut for Constructing Sentences

If the `|'==>'|` notation looks ugly to you, you can use the function `expr` instead:

In [ ]:
expr('~(P & Q)  ==>  (~P | ~Q)')

`expr` takes a string as input, and parses it into an `Expr`. The string can contain arrow operators: `==>`, `<==`, or `<=>`, which are handled as if they were regular Python infix operators. And `expr` automatically defines any symbols, so you don't need to pre-define them:

In [ ]:
expr('sqrt(b ** 2 - 4 * a * c)')

For now that's all you need to know about `expr`. If you are interested, we explain the messy details of how `expr` is implemented and how `|'==>'|` is handled in the appendix.

## Propositional Knowledge Bases: `PropKB`

A **Knowledge Base (KB)** is like a database that stores logical facts and rules. The `PropKB` class represents a knowledge base containing propositional logic sentences (statements that are either true or false).

### Understanding the Design Pattern
The base `KB` class has four main methods besides `__init__`. Here's an important design detail: the `ask` method simply calls the `ask_generator` method. This means when you create your own KB class, you only need to implement `ask_generator` - the `ask` method is already handled for you!

### Key Methods of `PropKB`:

**🏗️ Constructor: `__init__(self, sentence=None)`**
- Creates a `clauses` field - a list storing all sentences in the knowledge base
- Each sentence is stored as a 'clause' (a sentence made up of only literals connected by `or` operations)
- Think of clauses as the "building blocks" of logical knowledge

**📝 Adding Knowledge: `tell(self, sentence)`** 
- Use this to add new facts or rules to your knowledge base
- **Magic feature**: You can add ANY logical sentence - the method automatically converts it to CNF (Conjunctive Normal Form) and extracts clauses
- No need to manually convert to clauses - just tell the KB what you know!

**❓ Querying Knowledge: `ask_generator(self, query)`**
- This is the "brain" of the KB - it determines if the KB entails (logically implies) your query
- Uses the `tt_entails` function (truth table method) to check entailment
- **Return values explained**:
  - Returns `{}` (empty dict) if the query is entailed by the KB
  - Returns `None` if the query is not entailed
  - *Why not just True/False?* This design maintains consistency with First-Order Logic, where we need to return variable substitutions
- **Practical tip**: Use `ask_if_true()` if you prefer simple True/False answers

**🗑️ Removing Knowledge: `retract(self, sentence)`**
- Removes facts from the knowledge base
- Like `tell()`, it handles any sentence format - automatically converts to clauses before removal

## Wumpus World KB
Let us create a `PropKB` for the wumpus world with the sentences mentioned in `section 7.4.3`.

In [ ]:
wumpus_kb = PropKB()

We define the symbols we use in our clauses.<br/>
$P_{x, y}$ is true if there is a pit in `[x, y]`.<br/>
$B_{x, y}$ is true if the agent senses breeze in `[x, y]`.<br/>

In [ ]:
P11, P12, P21, P22, P31, B11, B21 = expr('P11, P12, P21, P22, P31, B11, B21')

Now we tell sentences based on `section 7.4.3`.<br/>
There is no pit in `[1,1]`.

In [ ]:
wumpus_kb.tell(~P11)

A square is breezy if and only if there is a pit in a neighboring square. This has to be stated for each square but for now, we include just the relevant squares.

In [ ]:
wumpus_kb.tell(B11 | '<=>' | ((P12 | P21)))
wumpus_kb.tell(B21 | '<=>' | ((P11 | P22 | P31)))

Now we include the breeze percepts for the first two squares leading up to the situation in `Figure 7.3(b)`

In [ ]:
wumpus_kb.tell(~B11)
wumpus_kb.tell(B21)

We can check the clauses stored in a `KB` by accessing its `clauses` variable

In [ ]:
wumpus_kb.clauses

We see that the equivalence $B_{1, 1} \iff (P_{1, 2} \lor P_{2, 1})$ was automatically converted to two implications which were inturn converted to CNF which is stored in the `KB`.<br/>
$B_{1, 1} \iff (P_{1, 2} \lor P_{2, 1})$ was split into $B_{1, 1} \implies (P_{1, 2} \lor P_{2, 1})$ and $B_{1, 1} \Longleftarrow (P_{1, 2} \lor P_{2, 1})$.<br/>
$B_{1, 1} \implies (P_{1, 2} \lor P_{2, 1})$ was converted to $P_{1, 2} \lor P_{2, 1} \lor \neg B_{1, 1}$.<br/>
$B_{1, 1} \Longleftarrow (P_{1, 2} \lor P_{2, 1})$ was converted to $\neg (P_{1, 2} \lor P_{2, 1}) \lor B_{1, 1}$ which becomes $(\neg P_{1, 2} \lor B_{1, 1}) \land (\neg P_{2, 1} \lor B_{1, 1})$ after applying De Morgan's laws and distributing the disjunction.<br/>
$B_{2, 1} \iff (P_{1, 1} \lor P_{2, 2} \lor P_{3, 2})$ is converted in similar manner.

## Knowledge based agents

A knowledge-based agent is a simple generic agent that maintains and handles a knowledge base.
The knowledge base may initially contain some background knowledge.
<br>
The purpose of a KB agent is to provide a level of abstraction over knowledge-base manipulation and is to be used as a base class for agents that work on a knowledge base.
<br>
Given a percept, the KB agent adds the percept to its knowledge base, asks the knowledge base for the best action, and tells the knowledge base that it has in fact taken that action.
<br>
Our implementation of `KB-Agent` is encapsulated in a class `KB_AgentProgram` which inherits from the `KB` class.
<br>
Let's have a look.

In [ ]:
psource(KB_AgentProgram)

The helper functions `make_percept_sentence`, `make_action_query` and `make_action_sentence` are all aptly named and as expected,
`make_percept_sentence` makes first-order logic sentences about percepts we want our agent to receive,
`make_action_query` asks the underlying `KB` about the action that should be taken and
`make_action_sentence` tells the underlying `KB` about the action it has just taken.

## Inference in Propositional Knowledge Base

**The Big Question**: Given what we know (our KB), can we conclude that some new statement α is true?

In formal logic terms: Does $\text{KB} \vDash \alpha$ (read as "KB entails α")?

This section explores several algorithms to answer this fundamental question in AI reasoning.

### Method 1: Truth Table Enumeration

**The Brute Force Approach** 🔍

This is the most straightforward method - it's like checking every possible scenario to see if our conclusion holds.

**How it works**:
1. **List all variables**: Identify all $n$ propositional symbols in the KB and query
2. **Generate all models**: Create all $2^n$ possible truth value assignments (this is why it's "brute force")
3. **Filter valid models**: Keep only models where the KB is true
4. **Check the query**: For each valid model, check if α is also true

**Think of it like this**: Imagine you have 3 light switches (A, B, C). There are $2^3 = 8$ ways to set them (on/off). Truth table enumeration checks all 8 combinations to see which ones satisfy your knowledge base, then verifies if your query is true in all those valid combinations.

**Trade-off**: This method is guaranteed to work but can be very slow for large numbers of variables (exponential complexity).

In [ ]:
psource(tt_check_all)

### Understanding the Truth Table Algorithm Step by Step

The algorithm essentially builds the complete truth table for $KB \implies \alpha$ and verifies it's always true.

**Step-by-Step Process**:

1. **Generate All Possible Worlds**: The algorithm recursively creates every possible combination of truth values for all symbols
2. **Filter Relevant Models**: For each combination (called a "model"), it checks if the model makes the KB true
3. **Test the Query**: For models where KB is true, it checks if α is also true

**The Core Logic Check**:
```
For each model: pl_true(kb, model) => pl_true(alpha, model)
```

**Key Insight**: This is equivalent to checking that KB and ¬α are never both true simultaneously:
```
NOT(pl_true(kb, model) & ~pl_true(alpha, model))
```

**Why This Works**: If we can find even one case where KB is true but α is false, then KB doesn't entail α. If no such case exists, then KB entails α.

**Function Roles**:
- `tt_check_all()`: Does the heavy lifting - generates models and checks the logical relationship
- `tt_entails()`: A convenient wrapper that extracts symbols and calls `tt_check_all()` with proper parameters

**Real-World Analogy**: Think of a detective checking an alibi. They need to verify that in every scenario where the witness's story is true, the suspect's claimed location is also true. If there's any scenario where the story holds but the alibi doesn't, the alibi fails.

In [ ]:
psource(tt_entails)

Keep in mind that for two symbols P and Q, P => Q is false only when P is `True` and Q is `False`.
Example usage of `tt_entails()`:

In [ ]:
tt_entails(P & Q, Q)

P & Q is True only when both P and Q are True. Hence, (P & Q) => Q is True

In [ ]:
tt_entails(P | Q, Q)

In [ ]:
tt_entails(P | Q, P)

If we know that P | Q is true, we cannot infer the truth values of P and Q. 
Hence (P | Q) => Q is False and so is (P | Q) => P.

In [ ]:
(A, B, C, D, E, F, G) = symbols('A, B, C, D, E, F, G')
tt_entails(A & (B | C) & D & E & ~(F | G), A & D & E & ~F & ~G)

We can see that for the KB to be true, A, D, E have to be True and F and G have to be False.
Nothing can be said about B or C.

Coming back to our problem, note that `tt_entails()` takes an `Expr` which is a conjunction of clauses as the input instead of the `KB` itself. 
You can use the `ask_if_true()` method of `PropKB` which does all the required conversions. 
Let's check what `wumpus_kb` tells us about $P_{1, 1}$.

In [ ]:
wumpus_kb.ask_if_true(~P11), wumpus_kb.ask_if_true(P11)

Looking at Figure 7.9 we see that in all models in which the knowledge base is `True`, $P_{1, 1}$ is `False`. It makes sense that `ask_if_true()` returns `True` for $\alpha = \neg P_{1, 1}$ and `False` for $\alpha = P_{1, 1}$. This begs the question, what if $\alpha$ is `True` in only a portion of all models. Do we return `True` or `False`? This doesn't rule out the possibility of $\alpha$ being `True` but it is not entailed by the `KB` so we return `False` in such cases. We can see this is the case for $P_{2, 2}$ and $P_{3, 1}$.

In [ ]:
wumpus_kb.ask_if_true(~P22), wumpus_kb.ask_if_true(P22)

### Method 2: Proof by Resolution

**The Contradiction Strategy** 🎯

Instead of checking all possibilities like truth tables, resolution uses a clever **proof by contradiction** approach.

**The Core Idea**: 
To prove that KB entails α, we assume α is false and try to derive a contradiction. If we succeed, then α must be true!

**Mathematical Foundation**:
- We want to check if $\text{KB} \vDash \alpha$ (KB entails α)
- **Key insight**: This is equivalent to checking if $\text{KB} \land \neg \alpha$ is unsatisfiable
- If KB and ¬α cannot both be true, then whenever KB is true, α must also be true

**The Resolution Algorithm**:

1. **Setup**: Add ¬α to the KB (assume the opposite of what we want to prove)
2. **Apply Resolution Rule**: Repeatedly combine clauses that have complementary literals
   - Example: From $(A \lor B)$ and $(\neg A \lor C)$, derive $(B \lor C)$
   - This eliminates the contradictory pair $(A, \neg A)$
3. **Termination Conditions**:
   - **Success**: If we derive the **empty clause** (□), we have a contradiction! → KB entails α
   - **Failure**: If no new clauses can be derived → KB does not entail α

**Why the Empty Clause Means Contradiction**:
The empty clause (□) represents "False" and can only arise from resolving complementary unit clauses like $P$ and $\neg P$. Since nothing can be both true and false simultaneously, this proves our assumption (¬α) leads to contradiction.

**Analogy**: It's like a logic puzzle where you assume the opposite of your conclusion and keep applying logical rules until you either:
- Reach an impossible situation (contradiction) → your original conclusion is correct
- Get stuck without finding impossibility → your conclusion doesn't follow from the given facts

There is  one catch however, the algorithm that implements proof by resolution cannot handle complex sentences. 
Implications and bi-implications have to be simplified into simpler clauses. 
We already know that *every sentence of a propositional logic is logically equivalent to a conjunction of clauses*.
We will use this fact to our advantage and simplify the input sentence into the **conjunctive normal form** (CNF) which is a conjunction of disjunctions of literals.
For eg:
<br>
$$(A\lor B)\land (\neg B\lor C\lor\neg D)\land (D\lor\neg E)$$
This is equivalent to the POS (Product of sums) form in digital electronics.
<br>
Here's an outline of how the conversion is done:
1. Convert bi-implications to implications
<br>
$\alpha\iff\beta$ can be written as $(\alpha\implies\beta)\land(\beta\implies\alpha)$
<br>
This also applies to compound sentences
<br>
$\alpha\iff(\beta\lor\gamma)$ can be written as $(\alpha\implies(\beta\lor\gamma))\land((\beta\lor\gamma)\implies\alpha)$
<br>
2. Convert implications to their logical equivalents
<br>
$\alpha\implies\beta$ can be written as $\neg\alpha\lor\beta$
<br>
3. Move negation inwards
<br>
CNF requires atomic literals. Hence, negation cannot appear on a compound statement.
De Morgan's laws will be helpful here.
<br>
$\neg(\alpha\land\beta)\equiv(\neg\alpha\lor\neg\beta)$
<br>
$\neg(\alpha\lor\beta)\equiv(\neg\alpha\land\neg\beta)$
<br>
4. Distribute disjunction over conjunction
<br>
Disjunction and conjunction are distributive over each other.
Now that we only have conjunctions, disjunctions and negations in our expression, 
we will distribute disjunctions over conjunctions wherever possible as this will give us a sentence which is a conjunction of simpler clauses, 
which is what we wanted in the first place.
<br>
We need a term of the form
<br>
$(\alpha_{1}\lor\alpha_{2}\lor\alpha_{3}...)\land(\beta_{1}\lor\beta_{2}\lor\beta_{3}...)\land(\gamma_{1}\lor\gamma_{2}\lor\gamma_{3}...)\land...$
<br>
<br>
The `to_cnf` function executes this conversion using helper subroutines.

In [ ]:
psource(to_cnf)

`to_cnf` calls three subroutines.
<br>
`eliminate_implications` converts bi-implications and implications to their logical equivalents.
<br>
`move_not_inwards` removes negations from compound statements and moves them inwards using De Morgan's laws.
<br>
`distribute_and_over_or` distributes disjunctions over conjunctions.
<br>
Run the cell below for implementation details.

In [ ]:
psource(eliminate_implications)
psource(move_not_inwards)
psource(distribute_and_over_or)

Let's convert some sentences to see how it works


In [ ]:
A, B, C, D = expr('A, B, C, D')
to_cnf(A |'<=>'| B)

In [ ]:
to_cnf(A |'<=>'| (B & C))

In [ ]:
to_cnf(A & (B | (C & D)))

In [ ]:
to_cnf((A |'<=>'| ~B) |'==>'| (C | ~D))

Coming back to our resolution problem, we can see how the `to_cnf` function is utilized here

In [ ]:
psource(pl_resolution)

In [ ]:
pl_resolution(wumpus_kb, ~P11), pl_resolution(wumpus_kb, P11)

In [ ]:
pl_resolution(wumpus_kb, ~P22), pl_resolution(wumpus_kb, P22)

### Method 3: Forward and Backward Chaining

**The Smart and Efficient Approach** ⚡

This is our third method for checking entailment, but with a **special restriction**: it only works with **Horn clauses**. The trade-off? It's much more efficient than the previous methods!

#### Understanding Horn Clauses

**What are Horn Clauses?**
Horn clauses are a special, restricted form of logical sentences that have **at most one positive literal**.

**Types of Horn Clauses**:
1. **Facts**: Single positive literals (e.g., `A`)
2. **Rules**: Implications with positive conclusion (e.g., `A ∧ B ⇒ C`)
3. **Constraints**: All negative literals (e.g., `¬A ∨ ¬B ∨ ¬C`)

**Example Horn Clause Transformation**:
```
Original: ¬a ∨ ¬b ∨ ¬c ∨ ¬d ∨ z
Using De Morgan's laws: a ∧ b ∧ c ∧ d ⇒ z
```

**Why Horn Clauses Are Special**:

🧠 **Human-Like Reasoning**: Horn clauses naturally represent how we think:
- "If it's raining AND I don't have an umbrella, THEN I'll get wet"
- This matches the pattern: `premises ⇒ conclusion`

⚡ **Computational Efficiency**: 
- Entailment checking is **linear** in the size of the knowledge base
- Each clause is processed at most once (unlike exponential methods)

🏗️ **Clear Structure**:
- **Body (premise)**: Conjunction of positive literals (the "if" part)
- **Head (conclusion)**: Single positive literal (the "then" part)
- **Facts**: Just the head with no body

**Key Advantages**:
1. **Predictable Performance**: No exponential explosion like truth tables
2. **Intuitive Rules**: Easy to understand and write
3. **Efficient Algorithms**: Forward and backward chaining work beautifully

**Implementation Note**: 
The `pl_fc_entails` function uses a specialized `PropDefiniteKB` class (extends `PropKB`) that includes helper methods for efficiently finding clauses with specific premises - this optimization makes chaining algorithms much faster.

In [ ]:
psource(PropDefiniteKB.clauses_with_premise)

Let's now have a look at the `pl_fc_entails` algorithm.

In [ ]:
psource(pl_fc_entails)

The function accepts a knowledge base `KB` (an instance of `PropDefiniteKB`) and a query `q` as inputs.
<br>
<br>
`count` initially stores the number of symbols in the premise of each sentence in the knowledge base.
<br>
The `conjuncts` helper function separates a given sentence at conjunctions.
<br>
`inferred` is initialized as a *boolean* defaultdict. 
This will be used later to check if we have inferred all premises of each clause of the agenda.
<br>
`agenda` initially stores a list of clauses that the knowledge base knows to be true.
The `is_prop_symbol` helper function checks if the given symbol is a valid propositional logic symbol.
<br>
<br>
We now iterate through `agenda`, popping a symbol `p` on each iteration.
If the query `q` is the same as `p`, we know that entailment holds.
<br>
The agenda is processed, reducing `count` by one for each implication with a premise `p`.
A conclusion is added to the agenda when `count` reaches zero. This means we know all the premises of that particular implication to be true.
<br>
`clauses_with_premise` is a helpful method of the `PropKB` class.
It returns a list of clauses in the knowledge base that have `p` in their premise.
<br>
<br>
Now that we have an idea of how this function works, let's see a few examples of its usage, but we first need to define our knowledge base. We assume we know the following clauses to be true.

In [ ]:
clauses = ['(B & F)==>E', 
           '(A & E & F)==>G', 
           '(B & C)==>F', 
           '(A & B)==>D', 
           '(E & F)==>H', 
           '(H & I)==>J',
           'A', 
           'B', 
           'C']

We will now `tell` this information to our knowledge base.

In [ ]:
definite_clauses_KB = PropDefiniteKB()
for clause in clauses:
    definite_clauses_KB.tell(expr(clause))

We can now check if our knowledge base entails the following queries.

In [ ]:
pl_fc_entails(definite_clauses_KB, expr('G'))

In [ ]:
pl_fc_entails(definite_clauses_KB, expr('H'))

In [ ]:
pl_fc_entails(definite_clauses_KB, expr('I'))

In [ ]:
pl_fc_entails(definite_clauses_KB, expr('J'))

### Effective Propositional Model Checking

The previous segments elucidate the algorithmic procedure for model checking. 
In this segment, we look at ways of making them computationally efficient.
<br>
The problem we are trying to solve is conventionally called the _propositional satisfiability problem_, abbreviated as the _SAT_ problem.
In layman terms, if there exists a model that satisfies a given Boolean formula, the formula is called satisfiable.
<br>
The SAT problem was the first problem to be proven _NP-complete_.
The main characteristics of an NP-complete problem are:
- Given a solution to such a problem, it is easy to verify if the solution solves the problem.
- The time required to actually solve the problem using any known algorithm increases exponentially with respect to the size of the problem.
<br>
<br>
Due to these properties, heuristic and approximational methods are often applied to find solutions to these problems.
<br>
It is extremely important to be able to solve large scale SAT problems efficiently because 
many combinatorial problems in computer science can be conveniently reduced to checking the satisfiability of a propositional sentence under some constraints.
<br>
We will introduce two new algorithms that perform propositional model checking in a computationally effective way.
<br>


### 1. DPLL (Davis-Putnam-Logemann-Loveland) Algorithm

**Smart Backtracking with Pruning** 🌲

DPLL is like backtracking search but with intelligent shortcuts that dramatically reduce the search space. Think of it as "backtracking search with a brain"!

**Core Concept**: Instead of blindly trying all $2^n$ possible truth assignments, DPLL uses three key optimizations:

#### 🚀 **Optimization 1: Early Termination**
**The Idea**: Sometimes you can determine satisfiability without assigning values to all variables.

**Example**: 
- Formula: $(P \lor Q) \land (P \lor R)$
- If we set $P = \text{true}$, both clauses become true immediately
- No need to worry about Q and R values!

**Benefit**: Massive pruning of the search tree - entire branches are eliminated

#### 🎯 **Optimization 2: Pure Symbol Heuristic**  
**The Idea**: If a variable appears with only one polarity (all positive or all negative), assign it to make all its clauses true.

**Example**:
- Formula: $(P \lor \neg Q) \land (\neg Q \lor \neg R) \land (R \lor P)$
- $P$ appears only positive → set $P = \text{true}$
- $Q$ appears only negative → set $Q = \text{false}$
- These assignments can only help, never hurt!

**Benefit**: Free simplifications that reduce problem size

#### ⚡ **Optimization 3: Unit Propagation**
**The Idea**: If a clause has only one unassigned literal, that literal must be true.

**Example Process**:
1. Start with: $(P \lor Q), (\neg P \lor R), (\neg Q)$
2. Third clause forces: $Q = \text{false}$ 
3. First clause becomes: $(P)$ → forces $P = \text{true}$
4. Second clause becomes: $(\text{true} \lor R) = \text{true}$ ✓

**Chain Reaction**: One unit assignment often creates more unit clauses, leading to a cascade of forced assignments.

**Why DPLL is Powerful**:
- **Systematic**: Guarantees to find a solution if one exists
- **Efficient**: Smart pruning avoids exponential explosion in many practical cases
- **Foundation**: Basis for modern SAT solvers used in industry

**Real-World Impact**: DPLL and its descendants power everything from circuit design verification to software bug detection!

In [ ]:
psource(dpll)

The algorithm uses the ideas described above to check satisfiability of a sentence in propositional logic.
It recursively calls itself, simplifying the problem at each step. It also uses helper functions `find_pure_symbol` and `find_unit_clause` to carry out steps 2 and 3 above.
<br>
The `dpll_satisfiable` helper function converts the input clauses to _conjunctive normal form_ and calls the `dpll` function with the correct parameters.

In [ ]:
psource(dpll_satisfiable)

Let's see a few examples of usage.

In [ ]:
A, B, C, D = expr('A, B, C, D')

In [ ]:
dpll_satisfiable(A & B & ~C & D)

This is a simple case to highlight that the algorithm actually works.

In [ ]:
dpll_satisfiable((A & B) | (C & ~A) | (B & ~D))

If a particular symbol isn't present in the solution, 
it means that the solution is independent of the value of that symbol.
In this case, the solution is independent of A.

In [ ]:
dpll_satisfiable(A |'<=>'| B)

In [ ]:
dpll_satisfiable((A |'<=>'| B) |'==>'| (C & ~A))

In [ ]:
dpll_satisfiable((A | (B & C)) |'<=>'| ((A | B) & (A | C)))

### 2. WalkSAT Algorithm

**Local Search with Random Escapes** 🎲

WalkSAT takes a completely different approach from DPLL - instead of systematic search, it uses **local search** similar to hill climbing optimization algorithms.

**The Core Strategy**:
1. **Start with a random assignment** of truth values
2. **Iteratively improve** by flipping variable values
3. **Balance greed with randomness** to avoid getting stuck

#### 🎯 **The WalkSAT Process**

**Step 1: Pick an Unsatisfied Clause**
- Look at all clauses that are currently false
- Randomly select one to focus on

**Step 2: Choose a Variable to Flip**  
This is where the algorithm gets clever - it has two strategies:

- **🧠 Greedy Move** (probability 1-p): Pick the variable flip that minimizes the number of newly broken clauses
- **🎲 Random Move** (probability p): Pick any variable in the clause randomly

**Step 3: Flip and Repeat**
- Change the chosen variable's truth value
- Repeat until satisfied or max iterations reached

#### ⚖️ **The Randomness Trade-off**

**Why Mix Greedy + Random?**
- **Pure Greedy**: Fast improvement but gets stuck in local optima (like hill climbing)
- **Pure Random**: Avoids getting stuck but makes no progress
- **Mixed Strategy**: Best of both worlds!

**Parameter Tuning**:
- **High p (more random)**: Better at escaping local minima, slower convergence  
- **Low p (more greedy)**: Faster initial progress, higher chance of getting stuck

#### 🏃‍♂️ **Advantages of WalkSAT**
- **Speed**: Often much faster than complete algorithms like DPLL for satisfiable instances
- **Scalability**: Handles very large problems that would overwhelm systematic methods  
- **Simplicity**: Easy to implement and understand

#### ⚠️ **Limitations**
- **Incomplete**: Cannot prove unsatisfiability (might search forever)
- **No Guarantees**: May fail to find solutions even when they exist
- **Parameter Sensitive**: Performance depends on the randomness parameter p

**When to Use**: WalkSAT excels when you need to quickly find solutions to large, satisfiable problems and don't need to prove unsatisfiability.

In [ ]:
psource(WalkSAT)

The function takes three arguments:
<br>
1. The `clauses` we want to satisfy.
<br>
2. The probability `p` of randomly changing a symbol.
<br>
3. The maximum number of flips (`max_flips`) the algorithm will run for. If the clauses are still unsatisfied, the algorithm returns `None` to denote failure.
<br>
The algorithm is identical in concept to Hill climbing and the code isn't difficult to understand.
<br>
<br>
Let's see a few examples of usage.

In [ ]:
A, B, C, D = expr('A, B, C, D')

In [ ]:
WalkSAT([A, B, ~C, D], 0.5, 100)

This is a simple case to show that the algorithm converges.

In [ ]:
WalkSAT([A & B, A & C], 0.5, 100)

In [ ]:
WalkSAT([A & B, C & D, C & B], 0.5, 100)

In [ ]:
WalkSAT([A & B, C | D, ~(D | B)], 0.5, 1000)

This one doesn't give any output because WalkSAT did not find any model where these clauses hold. We can solve these clauses to see that they together form a contradiction and hence, it isn't supposed to have a solution.

One point of difference between this algorithm and the `dpll_satisfiable` algorithms is that both these algorithms take inputs differently. 
For WalkSAT to take complete sentences as input, 
we can write a helper function that converts the input sentence into conjunctive normal form and then calls WalkSAT with the list of conjuncts of the CNF form of the sentence.

In [ ]:
def WalkSAT_CNF(sentence, p=0.5, max_flips=10000):
    return WalkSAT(conjuncts(to_cnf(sentence)), 0, max_flips)

Now we can call `WalkSAT_CNF` and `DPLL_Satisfiable` with the same arguments.

In [ ]:
WalkSAT_CNF((A & B) | (C & ~A) | (B & ~D), 0.5, 1000)

It works!
<br>
Notice that the solution generated by WalkSAT doesn't omit variables that the sentence doesn't depend upon. 
If the sentence is independent of a particular variable, the solution contains a random value for that variable because of the stochastic nature of the algorithm.
<br>
<br>
Let's compare the runtime of WalkSAT and DPLL for a few cases. We will use the `%%timeit` magic to do this.

In [ ]:
sentence_1 = A |'<=>'| B
sentence_2 = (A & B) | (C & ~A) | (B & ~D)
sentence_3 = (A | (B & C)) |'<=>'| ((A | B) & (A | C))

In [ ]:
%%timeit
dpll_satisfiable(sentence_1)
dpll_satisfiable(sentence_2)
dpll_satisfiable(sentence_3)

In [ ]:
%%timeit
WalkSAT_CNF(sentence_1)
WalkSAT_CNF(sentence_2)
WalkSAT_CNF(sentence_3)

On an average, for solvable cases, `WalkSAT` is quite faster than `dpll` because, for a small number of variables, 
`WalkSAT` can reduce the search space significantly. 
Results can be different for sentences with more symbols though.
Feel free to play around with this to understand the trade-offs of these algorithms better.

### SATPlan

**Planning Through Logic: From Problem to SAT Solver** 🤖➡️🎯

This section demonstrates how to transform **planning problems** (like "How do I get from A to B?") into **logical satisfaction problems** that can be solved automatically.

#### 🧠 **The Core Insight**
Instead of using specialized planning algorithms, we can encode the entire planning problem as a logical formula and let a SAT solver find the solution!

#### 📋 **The SATPlan Recipe (3 Simple Steps)**

**Step 1: 🏗️ Construct the Logic Formula**
Build a single large logical sentence that captures:
- **Initial State**: What's true at the beginning? 
  - Example: `At(Robot, RoomA, time=0)`
- **Action Constraints**: What actions are possible and their effects?
  - Example: `Move(RoomA, RoomB, time=t) ⟹ At(Robot, RoomB, time=t+1)`  
- **Goal Achievement**: What must be true at the end?
  - Example: `At(Robot, RoomC, time=T)`

**Step 2: 🔍 Feed to SAT Solver**
- Give the complete logical formula to any SAT solver (like DPLL or WalkSAT)
- The solver finds truth assignments that satisfy ALL constraints

**Step 3: 📜 Extract the Plan**
- Look at which action variables are assigned `True` in the solution
- These actions, in temporal order, form your plan!
- Example: `Move(A,B,t=0)=True, Move(B,C,t=1)=True` ⟹ Plan: [Move A→B, Move B→C]

#### ✨ **Why This Is Brilliant**
- **Leverages existing tools**: No need to write custom planning algorithms
- **Handles complex constraints**: Time, resources, conditional effects
- **Optimal solutions**: Can easily add cost constraints for optimal planning
- **Proven correctness**: If SAT solver finds solution, plan is guaranteed to work

Let's see this approach in action with concrete examples...

In [ ]:
psource(SAT_plan)

Let's see few examples of its usage. First we define a transition and then call `SAT_plan`.

In [ ]:
transition = {'A': {'Left': 'A', 'Right': 'B'},
            'B': {'Left': 'A', 'Right': 'C'},
            'C': {'Left': 'B', 'Right': 'C'}}


print(SAT_plan('A', transition, 'C', 2)) 
print(SAT_plan('A', transition, 'B', 3))
print(SAT_plan('C', transition, 'A', 3))

Let us do the same for another transition.

In [ ]:
transition = {(0, 0): {'Right': (0, 1), 'Down': (1, 0)},
            (0, 1): {'Left': (1, 0), 'Down': (1, 1)},
            (1, 0): {'Right': (1, 0), 'Up': (1, 0), 'Left': (1, 0), 'Down': (1, 0)},
            (1, 1): {'Left': (1, 0), 'Up': (0, 1)}}


print(SAT_plan((0, 0), transition, (1, 1), 4))

## First-Order Logic Knowledge Bases: `FolKB`

The class `FolKB` can be used to represent a knowledge base of First-order logic sentences. You would initialize and use it the same way as you would for `PropKB` except that the clauses are first-order definite clauses. We will see how to write such clauses to create a database and query them in the following sections.

## Criminal KB: A Complete First-Order Logic Example

**Learning by Example**: Let's build a knowledge base from a real scenario to see First-Order Logic in action!

### 📖 **The Scenario**
*"The law says that it is a crime for an American to sell weapons to hostile nations. The country Nono, an enemy of America, has some missiles, and all of its missiles were sold to it by Colonel West, who is American."*

### 🎯 **Our Goal**  
Transform this English text into First-Order Logic clauses that a computer can reason with.

### 🔍 **Knowledge Extraction Process**
This is one of the most challenging aspects of AI - converting natural language into formal logic. While automated extraction is an active research area, we'll do it step-by-step manually to understand the process.

**Why This Example Matters**:
- Shows the full pipeline from natural language → formal logic → automated reasoning
- Demonstrates how complex real-world knowledge can be represented
- Illustrates the power of logical inference (we'll prove West is a criminal!)

**What We'll Learn**:
1. How to identify key predicates and relationships
2. How to handle quantifiers (∀, ∃) in practical scenarios  
3. How existential instantiation works (introducing new constants)
4. How First-Order Logic captures relationships that propositional logic cannot

Let's start building our knowledge base step by step...

In [ ]:
clauses = []

<em>“... it is a crime for an American to sell weapons to hostile nations”</em><br/>
The keywords to look for here are 'crime', 'American', 'sell', 'weapon' and 'hostile'. We use predicate symbols to make meaning of them.

* `Criminal(x)`: `x` is a criminal
* `American(x)`: `x` is an American
* `Sells(x ,y, z)`: `x` sells `y` to `z`
* `Weapon(x)`: `x` is a weapon
* `Hostile(x)`: `x` is a hostile nation

Let us now combine them with appropriate variable naming to depict the meaning of the sentence. The criminal `x` is also the American `x` who sells weapon `y` to `z`, which is a hostile nation.

$\text{American}(x) \land \text{Weapon}(y) \land \text{Sells}(x, y, z) \land \text{Hostile}(z) \implies \text{Criminal} (x)$

In [ ]:
clauses.append(expr("(American(x) & Weapon(y) & Sells(x, y, z) & Hostile(z)) ==> Criminal(x)"))

<em>"The country Nono, an enemy of America"</em><br/>
We now know that Nono is an enemy of America. We represent these nations using the constant symbols `Nono` and `America`. the enemy relation is show using the predicate symbol `Enemy`.

$\text{Enemy}(\text{Nono}, \text{America})$

In [ ]:
clauses.append(expr("Enemy(Nono, America)"))

<em>"Nono ... has some missiles"</em><br/>
This states the existence of some missile which is owned by Nono. $\exists x \text{Owns}(\text{Nono}, x) \land \text{Missile}(x)$. We invoke existential instantiation to introduce a new constant `M1` which is the missile owned by Nono.

$\text{Owns}(\text{Nono}, \text{M1}), \text{Missile}(\text{M1})$

In [ ]:
clauses.append(expr("Owns(Nono, M1)"))
clauses.append(expr("Missile(M1)"))

<em>"All of its missiles were sold to it by Colonel West"</em><br/>
If Nono owns something and it classifies as a missile, then it was sold to Nono by West.

$\text{Missile}(x) \land \text{Owns}(\text{Nono}, x) \implies \text{Sells}(\text{West}, x, \text{Nono})$

In [ ]:
clauses.append(expr("(Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)"))

<em>"West, who is American"</em><br/>
West is an American.

$\text{American}(\text{West})$

In [ ]:
clauses.append(expr("American(West)"))

We also know, from our understanding of language, that missiles are weapons and that an enemy of America counts as “hostile”.

$\text{Missile}(x) \implies \text{Weapon}(x), \text{Enemy}(x, \text{America}) \implies \text{Hostile}(x)$

In [ ]:
clauses.append(expr("Missile(x) ==> Weapon(x)"))
clauses.append(expr("Enemy(x, America) ==> Hostile(x)"))

Now that we have converted the information into first-order definite clauses we can create our first-order logic knowledge base.

In [ ]:
crime_kb = FolKB(clauses)

The `subst` helper function substitutes variables with given values in first-order logic statements.
This will be useful in later algorithms.
It's implementation is quite simple and self-explanatory.

In [ ]:
psource(subst)

Here's an example of how `subst` can be used.

In [ ]:
subst({x: expr('Nono'), y: expr('M1')}, expr('Owns(x, y)'))

## Inference in First-Order Logic
In this section we look at a forward chaining and a backward chaining algorithm for `FolKB`. Both aforementioned algorithms rely on a process called <strong>unification</strong>, a key component of all first-order inference algorithms.

### Unification: Making Different Expressions Match

**The Pattern Matching Problem** 🧩

Imagine you have two puzzle pieces with slightly different patterns, but you suspect they might match if you fill in some blanks correctly. Unification is the algorithm that figures out how to fill those blanks!

**What is Unification?**
Unification finds **substitutions** (variable replacements) that make two logical expressions identical.

**Input**: Two logical expressions that might have variables
**Output**: A **unifier** - a dictionary of variable substitutions, or `None` if no match is possible

**How It Works**:
1. **Compare structures**: Check if the expressions have the same logical structure
2. **Match constants**: Constants must match exactly  
3. **Bind variables**: Variables can be replaced with constants or other terms
4. **Recursive matching**: Apply the same process to sub-expressions

**Key Principle**: A variable can unify with any term (constant, variable, or complex expression), but constants can only unify with identical constants.

**Why Unification Matters**:
- **Essential for FOL inference**: Both forward and backward chaining rely heavily on unification
- **Pattern matching**: Allows general rules to apply to specific situations
- **Variable binding**: Determines how abstract knowledge applies to concrete cases

**Example Walkthrough**:
- `unify(Cat(x), Cat(Fluffy))` → `{x: Fluffy}`
- Interpretation: "If x is Fluffy, then Cat(x) becomes Cat(Fluffy)"

Let's see unification in action with some examples...

In [ ]:
unify(expr('x'), 3)

In [ ]:
unify(expr('A(x)'), expr('A(B)'))

In [ ]:
unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(y)'))

In cases where there is no possible substitution that unifies the two sentences the function return `None`.

In [ ]:
print(unify(expr('Cat(x)'), expr('Dog(Dobby)')))

We also need to take care we do not unintentionally use the same variable name. Unify treats them as a single variable which prevents it from taking multiple value.

In [ ]:
print(unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(x)')))

### Forward Chaining Algorithm: Reasoning Forward from Facts

**The "What Can We Conclude?" Approach** 🔍➡️

Forward chaining starts with known facts and repeatedly applies rules to derive new facts until it can answer the query.

#### 🎯 **The Forward Chaining Process**

**Step 1: Start with Known Facts**
- Begin with all the facts explicitly stored in the KB
- These are our "seeds" for generating new knowledge

**Step 2: Apply Rules Systematically**
- For each rule in the KB: `premises ⟹ conclusion`
- Try to **unify** each premise with facts in the KB
- If ALL premises can be unified (matched), then we can conclude the rule's conclusion

**Step 3: Add New Conclusions**
- Apply the substitutions from unification to the conclusion
- Add this new fact to the KB (if it's not already there)

**Step 4: Repeat Until Done**
- Keep applying rules to derive new facts
- Stop when either:
  - 🎉 The query can be answered (we found a unifying substitution)
  - 😕 No new facts can be derived (query fails)

#### 🔄 **Why It's Called "Forward"**
- **Direction**: Facts → Rules → New Facts → More Rules → ...
- **Data-driven**: "Given what we know, what else can we conclude?"
- **Breadth-first feel**: Systematically explores all possible immediate conclusions

#### 💡 **Key Insight**
The `fol_fc_ask` function is a **generator** - it doesn't just find one answer, it finds ALL possible ways the query can be satisfied. This is crucial in FOL where there might be multiple valid substitutions.

**Real-World Analogy**: Like a detective who starts with evidence and systematically applies investigation procedures to uncover new leads, continuing until they can solve the case (or exhaust all leads).

In [ ]:
psource(fol_fc_ask)

Let's find out all the hostile nations. Note that we only told the `KB` that Nono was an enemy of America, not that it was hostile.

In [ ]:
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

The generator returned a single substitution which says that Nono is a hostile nation. See how after adding another enemy nation the generator returns two substitutions.

In [ ]:
crime_kb.tell(expr('Enemy(JaJa, America)'))
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

<strong><em>Note</em>:</strong> `fol_fc_ask` makes changes to the `KB` by adding sentences to it.

### Backward Chaining Algorithm: Reasoning Backward from Goals

**The "How Can We Prove This?" Approach** 🎯⬅️

Backward chaining starts with the goal (what we want to prove) and works backward through rules to find supporting facts.

#### 🎯 **The Backward Chaining Process**

**Step 1: Start with the Goal**
- Begin with the query we want to prove
- This becomes our initial "goal to satisfy"

**Step 2: Find Relevant Rules**  
- Look for rules that could prove the goal: `premises ⟹ goal`
- Try to **unify** the rule's conclusion with our current goal
- This gives us new subgoals: the rule's premises

**Step 3: Recursively Solve Subgoals**
- For each premise in the rule, treat it as a new goal
- Recursively apply backward chaining to prove each premise
- If ALL premises can be proven, then the original goal is proven

**Step 4: Try Alternative Paths**
- If one rule fails, try other rules that could prove the goal
- Success requires proving just ONE complete path back to facts

#### 🌳 **AND/OR Tree Structure**

**OR Nodes** (Multiple ways to prove a goal):
- "I can prove Criminal(x) using Rule A OR Rule B OR Rule C..."
- Only need ONE successful path

**AND Nodes** (All premises must be satisfied):  
- "To use this rule, I need to prove Premise₁ AND Premise₂ AND Premise₃..."
- Need ALL premises to succeed

#### 🔄 **Why It's Called "Backward"**
- **Direction**: Goal → Rules → Subgoals → Facts
- **Goal-driven**: "To prove this, what do I need to show?"
- **Depth-first feel**: Pursues one line of reasoning completely before trying alternatives

#### 💡 **Efficiency Advantage**
Backward chaining is often more efficient than forward chaining because it only explores reasoning paths that are relevant to answering the specific query.

**Real-World Analogy**: Like a lawyer building a case - they start with what they want to prove and work backward to identify what evidence and legal precedents they need to support their argument.

#### OR
The <em>OR</em> part of the algorithm comes from our choice to select any clause of the form $\text{lhs} \implies \text{goal}$. Looking at all rules's `lhs` whose `rhs` unify with the `goal`, we yield a substitution which proves all the conjuncts in the `lhs`. We use `parse_definite_clause` to attain `lhs` and `rhs` from a clause of the form $\text{lhs} \implies \text{rhs}$. For atomic facts the `lhs` is an empty list.

In [ ]:
psource(fol_bc_or)

#### AND
The <em>AND</em> corresponds to proving all the conjuncts in the `lhs`. We need to find a substitution which proves each <em>and</em> every clause in the list of conjuncts.

In [ ]:
psource(fol_bc_and)

Now the main function `fl_bc_ask` calls `fol_bc_or` with substitution initialized as empty. The `ask` method of `FolKB` uses `fol_bc_ask` and fetches the first substitution returned by the generator to answer query. Let's query the knowledge base we created from `clauses` to find hostile nations.

In [ ]:
# Rebuild KB because running fol_fc_ask would add new facts to the KB
crime_kb = FolKB(clauses)

In [ ]:
crime_kb.ask(expr('Hostile(x)'))

You may notice some new variables in the substitution. They are introduced to standardize the variable names to prevent naming problems as discussed in the [Unification section](#Unification)

## Appendix: The Implementation of `|'==>'|`

Consider the `Expr` formed by this syntax:

In [ ]:
P |'==>'| ~Q

What is the funny `|'==>'|` syntax? The trick is that "`|`" is just the regular Python or-operator, and so is exactly equivalent to this: 

In [ ]:
(P | '==>') | ~Q

In other words, there are two applications of or-operators. Here's the first one:

In [ ]:
P | '==>'

What is going on here is that the `__or__` method of `Expr` serves a dual purpose. If the right-hand-side is another `Expr` (or a number), then the result is an `Expr`, as in `(P | Q)`. But if the right-hand-side is a string, then the string is taken to be an operator, and we create a node in the abstract syntax tree corresponding to a partially-filled  `Expr`, one where we know the left-hand-side is `P` and the operator is `==>`, but we don't yet know the right-hand-side.

The `PartialExpr` class has an `__or__` method that says to create an `Expr` node with the right-hand-side filled in. Here we can see the combination of the `PartialExpr` with `Q` to create a complete `Expr`:

In [ ]:
partial = PartialExpr('==>', P) 
partial | ~Q

This clever [operator overloading trick](http://code.activestate.com/recipes/384122-infix-operators/) is due to [Ferdinand Jamitzky](http://code.activestate.com/recipes/users/98863/), with a modification by [C. G. Vedant](https://github.com/Chipe1), who suggested using a string inside the or-bars.

## Appendix: The Implementation of `expr` 

**Behind the Scenes: How `expr` Works Its Magic** 🎩✨

Ever wondered how `expr('P ==> Q')` magically becomes a proper `Expr` object? Here's the clever implementation:

#### 🔧 **The Two-Step Transformation Process**

**Step 1: 🔄 String Surgery**
- Replace logical operators with Python-compatible versions:
  - `"P ==> Q"` becomes `"P |'==>'| Q"`  
  - `"P <=> Q"` becomes `"P |'<=>'| Q"`
  - And so on for other logical operators

**Step 2: 🌍 Magical Evaluation Environment**
- Use Python's `eval()` function in a special environment
- In this environment, every identifier automatically becomes a Symbol
- So `P` becomes `Symbol('P')`, `Q` becomes `Symbol('Q')`, etc.

#### 💡 **The Result**
Both of these expressions are equivalent:

In [ ]:
expr('~(P & Q)  ==>  (~P | ~Q)')

is equivalent to doing:

In [ ]:
P, Q = symbols('P, Q')
~(P & Q)  |'==>'|  (~P | ~Q)

One thing to beware of: this puts `==>` at the same precedence level as `"|"`, which is not quite right. For example, we get this:

In [ ]:
P & Q  |'==>'|  P | Q

which is probably not what we meant; when in doubt, put in extra parens:

In [ ]:
(P & Q)  |'==>'|  (P | Q)

## Examples

In [ ]:
from notebook import Canvas_fol_bc_ask
canvas_bc_ask = Canvas_fol_bc_ask('canvas_bc_ask', crime_kb, expr('Criminal(x)'))

# Authors

This notebook by [Chirag Vartak](https://github.com/chiragvartak) and [Peter Norvig](https://github.com/norvig).

